<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2022/images/logo.png?raw=true" alt="2022年度ゲノム情報解析入門" height="100px" align="middle">

<div align="right"><a href="https://github.com/CropEvol/lecture#section2">実習表ページに戻る</a></div>

QTL-seq入門 (1): 参照配列（リファレンス配列）の準備
---

　今回から、ゲノム情報解析の一例として、次世代シーケンサーを使った遺伝子マッピング手法のひとつ、QTL-seqを勉強・体験していきます。

QTL-seq入門シリーズ: 
- QTL-seq入門 (1): 参照配列（リファレンス配列）の準備　←今回
- QTL-seq入門 (2): リードアライメント、SNP検出
- QTL-seq入門 (3): SNP-indexの算出、グラフ描画

# 今回の勉強内容
　このテキストでは、そのQTL-seq入門シリーズの第1回として、以下を勉強します。

1. QTL-seqとは？
  1. 遺伝子マッピング
  1. QTL-seqの概要
  1. QTL-seqの解析手順
1. 参照配列（リファレンス配列）を準備する
  1. 公開データを使う
  1. ゲノムアセンブル：自分でイチから作る
1. ゲノムアセンブル入門
  1. ゲノムアセンブルの仕組み
  1. リードの量や長さの影響

---

　一度次のコードセルを実行して、実習に必要なライブラリのインストールや、プログラム、サンプルファイルのダウンロードしてください。

In [ ]:
################################
##  以下の実習の前に一度実行してください。##
################################
## Biopythonのインストール、 プログラム・サンプルファイルのダウンロード
!pip install -q biopython
!wget -q -O 'assemble.py' https://raw.githubusercontent.com/CropEvol/lecture/master/textbook_2022/scripts/assemble.py
!wget -q -O 'mini_genome.fa' https://raw.githubusercontent.com/CropEvol/lecture/master/textbook_2022/datasets/mini_genome.fa
!wget -q -O 'mini_genome_with_repeat.fa' https://raw.githubusercontent.com/CropEvol/lecture/master/textbook_2022/datasets/mini_genome_with_repeat.fa
## 確認
!pip list | grep 'biopython'
!ls | grep -e 'assemble.py' -e 'mini_genome'

# 1. QTL-seqとは？

## 1-1. 遺伝子マッピング
　QTL-seqは**遺伝子マッピング（Gene mapping）**手法のひとつです。まず遺伝子マッピングとは何か、についてを少し学びましょう。

　遺伝子マッピングは、個体AとBのあいだの形質の違いを作っている遺伝子（原因遺伝子）がゲノム上のどこにあるのかを特定する手法です。どのようにおこなうのか？

1. 個体AとBのゲノム配列を比較すると、個体間の塩基の違い（例えば、SNP; Single nucleotide polymorphism）を発見できます。しかし、個体AとBの間の比較だけでは、どの変異が個体間の形質の違いを作っているのかを特定できません。
2. そこで、個体AとBのゲノムをランダムにシャッフルした個体集団（分離集団）を作ります。この集団は、個体AとBを交配することで得られます。
3. 分離集団の各個体の形質とゲノム上の多数のマーカーの遺伝子型も調べ、形質と遺伝子型に連関（Association）があるゲノム上の場所を探し、原因遺伝子を含むゲノム領域を特定します。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2022/images/gene_mapping.png?raw=true" alt="gene_mapping" height="280px" align="middle">

　上述のように、分離集団の全個体の遺伝子型を使って遺伝子マッピングする方法を**QTL解析（QTL analysis）**と言います。

　QTL-seqは、QTL解析をより省力的にした方法で、次世代シーケンスデータを用いて、分離集団の一部の個体のシーケンス情報から遺伝子マッピングをおこないます。

## 1-2. QTL-seqの概要

　QTL-seqでは、分離集団のうち極端な形質値をもつ個体のバルクシーケンスデータを用いて遺伝子マッピングをおこないます。バルクシーケンス中に含まれるアリルの割合（SNP-index）の推移をゲノム全体にわたって調べることで、遺伝子の位置を特定します。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2022/images/qtlseq_overview.png?raw=true" alt="QTL-seq" height="300px" align="middle">

> <small>Takagi, H., Abe, A., Yoshida, K., Kosugi, S., Natsume, S., Mitsuoka, C., Uemura, A., Utsushi, H., Tamiru, M., Takuno, S., Innan, H., Cano, L., Kamoun, S. & Terauchi, R. (2013). [QTL‐seq: rapid mapping of quantitative trait loci in rice by whole genome resequencing of DNA from two bulked populations](https://onlinelibrary.wiley.com/doi/full/10.1111/tpj.12105). The Plant Journal, 74(1), 174-183.</small>

- シーケンシングまで: 
  1. 形質が異なる2つの個体を交配し、分離集団を作る。
  1. 分離集団のうち、極端な形質値をもつ数十個体からDNAを抽出する。
  1. 形質値の低い個体のDNAをひとつにまとめ（プーリンングし）、バルクDNAを作る。同様に、形質値の高い個体のバルクDNAも作る（以下、それぞれを「Lowバルク」、「Highバルク」と呼ぶ）。
  1. それぞれのバルクDNAをシーケンシングし、リードを得る。

- シーケンシング以降: 
  1. Lowバルクのリードを片親（図では親A）のゲノム配列（参照配列）に対してアライメントする。
  1. アライメントされたリードの配列と参照配列の間を比較することで、SNPを検出する。SNPを検出することで次のことがわかる。
    - SNPの位置（SNP座）
    - 親A由来のSNPアリル（以下、Aアリル）の個数
    - 親B由来のSNPアリル（以下、Bアリル）の個数  
      <small>※ これらのアリルはバルクにした個体からランダムサンプリングされたアリルである</small>
  1. 各SNP座について、アリルの総数に対するBアリルの割合（以下、**SNP-index**）を調べる。
  1. Highバルクについても同様にSNP-indexを調べる。
  1. LowバルクとHighバルクのゲノム全域にわたるSNP-indexの分布を調べる。
  1. 原因遺伝子付近は、HighバルクではSNP-indexが高く（すなわち、Bアリルの割合が高く）、LowバルクではSNP-indexが低いことが期待される。そのようなゲノム領域を探しQTLとして同定する。
  1. また、HighバルクとLowバルクのSNP-indexの差分（delta SNP-index）を得ることで、ひとつのグラフでQTL同定をおこなうことも可能になる。

## 1-3. QTL-seqの解析手順

　シーケンスデータを得たあと、以下のような流れで解析します。

QTL-seqの流れ:
1. ゲノム配列（参照配列）を用意する
2. 参照配列にリードをアライメントする
3. SNPを検出する
4. SNP-indexを算出する
5. SNP-indexのグラフを描画する

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2022/images/qtlseq_workflow.png?raw=true" alt="QTL-seq workflow" height="180px" align="middle">

　今回、①のステップ、参照配列（リファレンス配列）をどのように準備するかを勉強していきましょう。

# 2. 参照配列（リファレンス配列）を準備する

## 2-1. 公開データを使う

　よく研究されている生物種であれば、データベースで公開されているゲノム配列を利用できます。最近では、多くの生物でゲノム解読が進んできているため、探してみるとゲノム配列が公開されていることもあります。
- NCBI: https://www.ncbi.nlm.nih.gov/genome
  - イネ Oryza sativa: https://www.ncbi.nlm.nih.gov/genome/?term=rice
- Ensemble: http://ensemblgenomes.org/


## 2-2. ゲノムアセンブル：自分でイチから作る

　あまり研究されていない生物種を対象にしている場合、ゲノム配列の決定（ゲノム解読）をおこなう必要があります。

　ゲノム解読にはそれなりに費用がかかりますが、近年のシーケンシング技術の進歩により、シーケンシングの費用は低くなってきました。数十〜数百Mbのゲノムの生物であれば、ひとつの研究室でも「ゲノム解読」をおこなえる時代です。

　ゲノム解読には、シーケンシングリード（以下、リード）が必要です。すぐ下のコードセルのプログラムを動かすと、あるゲノム配列の仮想リード100本（各4塩基長）を得られます。
- 出力ファイル: `reads.fa`

In [ ]:
from assemble import GetSubseq, Sequencing
genome = GetSubseq("mini_genome.fa")
Sequencing(genome, read_length=4, n_reads=100, verbose=True)

　先ほど得たリード（`reads.fa`）を使って、ゲノム配列の再構築（**ゲノムアセンブル**）を試みてみましょう。

　次のプログラムを実行すると、リードのアセンブルが自動的におこなわれます。

In [ ]:
from assemble import RunAssemble
assembled_genome = RunAssemble("reads.fa")
print(assembled_genome)

　再構築された配列が、リードの発生元の塩基配列と同じか確認してください。

手順:
1. 左ペイン「フォルダ」アイコンを開き、`mini_genome.fa`をダブルクリック等で開いてください。
1. `mini_genome.fa`に書かれている塩基配列と上でアセンブルした塩基配列が一致するか確認してください。

### FASTA形式ファイル

　先ほどのプログラム実行で作成されるファイル（`reads.fa`）は、**FASTA**と呼ばれる形式でリード配列の情報が書かれています。

　FASTA形式は、生物の塩基配列情報をデータとして保存しておくのに最もよく使われるフォーマットです。

```
----- FASTA形式 ---------
>塩基配列名
AATGC....（塩基配列本体）
-------------------------------
例)
>read_1
GTCT
>read_2
AAAC
```

# 3. ゲノムアセンブル入門

　上述の例では、リードのアセンブルを自動的におこないました。リードからどのようにゲノム配列を再構築しているのか？ ここからは、ゲノムアセンブルの工程をもう少し詳しく勉強していきましょう。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2022/images/reads_to_assembly.png?raw=true" alt="reads_to_assembly" height="200px" align="middle">

## 3-1. ゲノムアセンブルの仕組み

### 3-1-1. パスからゲノム配列へ
　あるゲノム配列から多数のリードを得たとしましょう。アセンブルではこのうち同じ配列を除去したユニークなリード配列セット(ここでは「重複のないリード」と呼ぶ)を使います。

　ゲノム配列がわかっている場合、重複のないリードを順番に並べることができ、それらの配列を繋ぐと元のゲノム配列を復元できます。このリードの繋ぎ方のことを**パス（Path）**と言います。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2022/images/reads_path_to_sequence.png?raw=true" alt="reads_path_to_sequence" height="250px" align="middle">

#### 問題1

　次のパスから得られるアセンブル配列は何でしょうか？

```
Path: 
ACT -> CTT -> TTT -> TTC -> TCT -> CTG
```

#### 解答1


```
Assembled sequence: ACTTTCTG
```

### 3-1-2. リードからパスへ

　リードの繋げ方（パス）がわかれば、それらを繋げてより長い配列にできることがわかりました。

　実際のゲノムアセンブルでは、元のゲノム配列の情報はないため、リードからパスを探す必要があります。次に、どのようにしてパスを見つけるのかを勉強していきましょう。

#### STEP1

　リードから前方と後方の数塩基（下図では2塩基）の部分配列を得ます。リードから得られた部分配列であるため、前方塩基から後方塩基の方向に矢印で繋ぐことができます。

　グラフ理論では、円で示されたものを**ノード（Node）**、矢印を**エッジ（Edge）**と呼びます。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2022/images/node_and_edge.png?raw=true" alt="node_and_edge" height="150px" align="middle">

#### 問題2

　次の6つのリード（各3塩基）について、それぞれの前方2塩基と後方2塩基の関係図（`前方2塩基 -> 後方2塩基`）を描いてください。

```
ACT, CTT, CTG, TCT, TTC, TTT

e.g.)
[ACT] AC -> CT
```

#### 解答2

```
[ACT] AC -> CT
[CTT] CT -> TT
[CTG] CT -> TG
[TCT] TC -> CT
[TTC] TT -> TC
[TTT] TT -> TT
```

#### STEP2

　多数の「前方塩基 -> 後方塩基」のうち、共通する部分配列のノードをそれぞれを一つにまとめていきます。このとき、部分配列同士を繋ぐ矢印（エッジ）情報は保持したままにします。

　このようにして得られた部分配列同士の関係のグラフは、**de Bruijnグラフ**と呼ばれています。このde Bruijnグラフの重要な点は、「エッジがリードの配列情報も保持している」ことです。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2022/images/nodes_edges_to_graph.png?raw=true" alt="nodes_edges_to_graph" height="250px" align="middle">

#### 問題3

　次の6つの前方・後方塩基の関係図からde Bruijnグラフを描いてください。


```
AC -> CT
CT -> TT
CT -> TG
TC -> CT
TT -> TC
TT -> TT
```

#### 解答3

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2022/images/nodes_edges_to_graph_quize.png?raw=true" alt="nodes_edges_to_graph_quize" height="250px" align="middle">

#### STEP3
 元のゲノム配列は、重複のないリードを一度ずつ使用し、繋ぐことで再構築できます。使用しないリードがあると、元配列より短いアセンブル配列が出来上がってしまいます。反対に、2回以上使用するリードがあると、過剰な配列をもったアセンブル配列が出来上がってしまします。つまり、de Bruijnグラフから「（重複のないリードを表している）各エッジを一度ずつ通るパスを見つける」と元のゲノム配列を再現できます。

　これは与えられたグラフから**一筆書きの経路（準オイラー路（semi-Eulerian trail）**を見つけることに相当します。与えられたグラフが「一筆書き可能な経路かどうか」や「一筆書きの経路を効率よく見つける方法（アルゴリズム）」はすでに知られており、重複のないリード配列情報からゲノムアセンブルをおこなうことができます。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2022/images/graph_to_sequence.png?raw=true" alt="graph_to_sequence" height="250px" align="middle">



#### 問題4

　問題3で作成したde Bruijnグラフから一筆書き経路を探してください。また、その経路から得られるゲノム配列は何でしょうか？

#### 解答4

```
Nodes order: AC -> CT -> TT -> TT -> TC -> CT -> TG
Edges order:   ACT ->CTT ->TTT ->TTC ->TCT ->CTG

Assembled sequence: ACTTTCTG
```

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2022/images/graph_to_sequence_quize.png?raw=true" alt="graph_to_sequence_quize" height="200px" align="middle">

#### ---

　ここまで見てきたように、ゲノムアセンブルでは、(1) リードからde Bruijnグラフを構築してパスを探し、(2) パスを長い配列に変換することでゲノム配列を再構築しています。

　では、リードを得れば常にゲノム配列を再構築できるのか？ 実際のところ、リードの量や長さ、ゲノムサイズの大きさ、ゲノム中の繰り返し配列や重複配列の存在などにより、完全なゲノム配列を再構築するのは難しいです。

　以降では、取得するリードの量や長さを変えて、ゲノムアセンブルをし、元のゲノム配列を再現できるか試してみましょう。
　

## 3-2. リードの量や長さの影響

　まず、20塩基のゲノム配列の例を見てみましょう。

```
# ゲノム配列
GTCTATTATGGAAAACACCA
```

　すぐ下のコードセルを動かすと、あるゲノム配列からシーケンシングをし、リードのアセンブルを試みるプログラムが実行されます。

In [ ]:
from assemble import GetSubseq, Sequencing, RunAssemble
genome = GetSubseq("mini_genome.fa")
Sequencing(genome, read_length=4, n_reads=100, verbose=False, draw=True)
assembled_genome = RunAssemble("reads.fa")
print("Original       : ", genome)
print("Assembled : ", assembled_genome)

### 3-2-1. リードの量が少ない場合
　プログラム中の`n_reads`の数字を「10」に変更して実行してください。

```python
# 変更後
Sequencing(..., read_length=4, n_reads=10, ...)
```

In [ ]:
## 先ほどと同じプログラムです。
from assemble import GetSubseq, Sequencing, RunAssemble
genome = GetSubseq("mini_genome.fa")
Sequencing(genome, read_length=4, n_reads=100, verbose=False, draw=True)
assembled_genome = RunAssemble("reads.fa")
print("Original       : ", genome)
print("Assembled : ", assembled_genome)

　リード数が少なすぎると、十分にオーバーラップしたリードを得られる可能性が少なくなります。そのため、アセンブル自体に失敗します。

### 3-2-2. リードの長さが短い場合
　　プログラム中の`read_length`の数字を「2」や「3」に変更して実行してください。

```python
# 変更後（2に変更した場合）
Sequencing(..., read_length=2, ...)
```

In [ ]:
## 先ほどと同じプログラムです。
from assemble import GetSubseq, Sequencing, RunAssemble
genome = GetSubseq("mini_genome.fa")
Sequencing(genome, read_length=4, n_reads=100, verbose=False, draw=True)
assembled_genome = RunAssemble("reads.fa")
print("Original       : ", genome)
print("Assembled : ", assembled_genome)

　リードが短すぎると、十分にオーバーラップしたリードを得られたとしても、de Bruijnグラフが複雑になり、一筆書きできないグラフが得られたり、一筆書きできたとしても、元の配列を再構築できるグラフではなかったりします。したがって、アセンブルに失敗するか、ミスアセンブル（元のゲノムとは異なる配列）になることが多いです。

### 3-3-3. 長い繰り返し配列のあるゲノム配列の場合
　次は、少し長くて（40bp）、繰り返し配列のある（`CTT`が4回連続である）ゲノムを使ってみましょう。

```
# ゲノム配列
ATCACCTCTTCTTCTTCTTCCCATTCATTCAGGAACTCGC
```

　下のプログラムは初期状態のまま実行してもアセンブルに失敗し、配列を得られないはずです。

　プログラムの3行目の`read_length`や`n_reads`の値を変更して、元のゲノムと同じ配列を再構築してみてください。

In [ ]:
from assemble import GetSubseq, Sequencing, RunAssemble
genome = GetSubseq("mini_genome_with_repeat.fa")
Sequencing(genome, read_length=4, n_reads=100, verbose=False, draw=True)
assembled_genome = RunAssemble("reads.fa")
print("Original       : ", genome)
print("Assembled : ", assembled_genome)

　長いリードが繰り返し配列のあるゲノムの再現に役立つことをみてきました。

　生物のゲノムには繰り返し配列や重複配列が多く存在しています。十分な量のショートリード（150bp程度）があったとしても、ショートリードだけでゲノム配列を再現するのはほぼ不可能です。現在のゲノム解読では、多数のショートリードだけでなく、量は少ないが非常に長いリード（1kb〜数Mb; ロングリード<sup>※</sup>）を組み合わせて、ゲノムアセンブルをおこなう方法が主流になっています。

<small><sup>※</sup> ロングリードを得るために、NanoporeやPacBio Sequel といったシーケンサーが利用されています。</small>


# まとめ

　今回、QTL-seqの概要と解析の流れ、その解析に最初のステップ「参照配列（リファレンス配列）の準備」をみてきました。
- QTL-seq: 
  - 個体間の形質の違いに関わる遺伝子のゲノム位置を特定する手法
- 参照配列（リファレンス配列）の準備:
  - 公開されているゲノム配列を利用する
  - ゲノム配列をイチから作る（ゲノムアセンブリ）
- ゲノムアセンブル: 
  - リードからゲノム配列を再構築する方法
  - リードの部分配列からde Bruijnグラフを作り、一筆書き経路を見つけ、配列を繋げる
  - リードのサイズや量はアセンブリに重要なファクター。特に、長いリード（ロングリード）は完全な配列を再現するのに役立つ


---

ゲノムアセンブリの基礎理論の参考Webページ:  
- [How do we assemble genomes?](https://www.bioinformaticsalgorithms.org/bioinformatics-chapter-3) (Bioinformatics Algorithms / Phillip Compeau)

<div align="right"><a href="https://github.com/CropEvol/lecture#section2">実習表ページに戻る</a></div>